# First Steps

## Creating a simple Kafka consumer app

In [ ]:
from IPython.display import Markdown as md

In [ ]:
import nest_asyncio

nest_asyncio.apply()

For our first demo we will create the simplest possible Kafka consumer and run it using uvicorn.

The consumer will:

    1. Connect to the Kafka Broker we setup in the Intro guide
    2. Listen to the hello topic
    3. Write any message received from the hello topic to stdout
    
To create the consumer, first, create a file named <b>hello_kafka_consumer.py</b> and copy the following code to it:

In [ ]:
#| hide_input
app_script ="""
from fast_kafka_api.application import FastKafkaAPI
from pydantic import BaseModel, Field
from os import environ

kafka_server_url = environ["KAFKA_HOSTNAME"]
kafka_server_port = environ["KAFKA_PORT"]

kafka_config = {
        "bootstrap_servers": f"{kafka_server_url}:{kafka_server_port}",
    }

class HelloKafkaMsg(BaseModel):
    msg: str = Field(
        ...,
        example="Hello",
        description="Demo hello world message",
    )

app = FastKafkaAPI(
        kafka_config=kafka_config
    )

@app.consumes()
async def on_hello(msg: HelloKafkaMsg):
    print(f"Got data, msg={msg.msg}")
"""

md(f"```python\n{app_script}\n```")

```python

from fast_kafka_api.application import FastKafkaAPI
from pydantic import BaseModel, Field
from os import environ

kafka_server_url = environ["KAFKA_HOSTNAME"]
kafka_server_port = environ["KAFKA_PORT"]

kafka_config = {
        "bootstrap_servers": f"{kafka_server_url}:{kafka_server_port}",
    }

class HelloKafkaMsg(BaseModel):
    msg: str = Field(
        ...,
        example="Hello",
        description="Demo hello world message",
    )

app = FastKafkaAPI(
        kafka_config=kafka_config
    )

@app.consumes()
async def on_hello(msg: HelloKafkaMsg):
    print(f"Got data, msg={msg.msg}")

```

To run this consumer, in your terminal, run:

In [ ]:
#|hide_input
run_cmd = "python3 -m uvicorn hello_kafka_consumer:app"

md(f"```shell\n{run_cmd}\n```")

```shell
python3 -m uvicorn hello_kafka_consumer:app
```

After running the command, you should see something simmilar to the ouput below:

In [ ]:
#| notest
import os

from tempfile import TemporaryDirectory
from pathlib import Path

with TemporaryDirectory() as d:
    consumer_script = Path(d)/"hello_kafka_consumer.py"
    with open(consumer_script, "a+") as file:
        file.write(app_script)
    os.chdir(d)
    !{run_cmd}

[INFO] fast_kafka_api._components.asyncapi: ok
INFO:     Started server process [15137]
INFO:     Waiting for application startup.
[INFO] fast_kafka_api._components.asyncapi: Async specifications generated at: 'asyncapi/spec/asyncapi.yml'
[INFO] fast_kafka_api._components.asyncapi: Async docs generated at 'asyncapi/docs'
[INFO] fast_kafka_api._components.asyncapi: Output of '$ npx -y -p @asyncapi/generator ag asyncapi/spec/asyncapi.yml @asyncapi/html-template -o asyncapi/docs --force-write'

Done! ✨
Check out your shiny new generated files at /tmp/tmpoqscot_4/asyncapi/docs.


[INFO] fast_kafka_api._components.aiokafka_consumer_loop: aiokafka_consumer_loop() starting..
[INFO] fast_kafka_api._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer created.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
[INFO] fast_kafka_api._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer started.
[INFO

Now you can interact with your consumer, by sending the messages to the subscribed 'hello' topic, don't worry, we will cover this in the next step of this guide.

## Sending first message to your consumer

After we have created and run our first consumer, we should send a message to it, to make sure it is working properly.

If you are using the Kafka setup as described in the Intro guide, you can follow the steps listed here to send a message to the hello topic.

First, connect to your running kafka broker by running:

``` shell
docker run -it kafka /bin/bash
```

Then, when connected to the container, run:

``` shell
kafka-console-producer.sh --bootstrap-server=localhost:9092 --topic=hello
```

This will open an interactive connection to the hello topic, now you can write your mesages to the topic and they will be consumed by our consumer.

In the shell, type:
``` shell
{"msg":"hello"}
```
and press enter. This will send a hello message to the topic which will be read by our running consumer and outputed to stdout.

Check the output of your consumer (terminal where you run the uvicorn command) and confirm that your consumer has read the Kafka message. You shoud see something like this:
``` shell
Got data, msg=hello
```

## Creating a hello producer

Consuming messages is only a part of this Library functionality, the other big part is producing the messages. So, let's create our first kafka producer which will send it's greetings to our consumer periodically.

The producer will:

    1. Connect to the Kafka Broker we setup in the Intro guide
    2. Connect to the hello topic
    3. Periodically send a message to the hello world topic
    
To create the producer, first, create a file named <b>hello_kafka_producer.py</b> and copy the following code to it:

```python
from fast_kafka_api.application import FastKafkaAPI
from fast_kafka_api.asyncapi import KafkaMessage
from pydantic import Field

kafka_config = {
        "bootstrap.servers": "localhost:9092",
        "group.id": "hello_producer_group"
    }

class HelloKafkaMsg(KafkaMessage):
    msg: str = Field(
        ...,
        example="Hello",
        description="Demo hello world message",
    )

app = FastKafkaAPI(
        kafka_config=kafka_config
    )

@app.produces
async def on_hello(msg: HelloKafkaMsg):
    return msg

while(True):
    sleep(2000)
    on_hello(HelloKafkaMsg(msg = "hello"))
```

To run this producer, in your terminal, run:

``` shell
python3 -m uvicorn hello_kafka_producer:app
```

After running the command, you should see something simmilar to the ouput below:

``` shell
[INFO] fast_kafka_api.asyncapi: ok
INFO:     Started server process [942]
INFO:     Waiting for application startup.
[INFO] fast_kafka_api.asyncapi: Async specifications generated at: 'asyncapi/spec/asyncapi.yml'
[INFO] fast_kafka_api.asyncapi: Async docs generated at 'asyncapi/docs'
[INFO] fast_kafka_api.asyncapi: Output of '$ npx -y -p @asyncapi/generator ag asyncapi/spec/asyncapi.yml @asyncapi/html-template -o asyncapi/docs --force-write'

Done! ✨
Check out your shiny new generated files at /work/fast-kafka-api/demos/asyncapi/docs.


%4|1670831653.286|CONFWARN|rdkafka#producer-1| [thrd:app]: Configuration property group.id is a consumer property and will be ignored by this producer instance
[INFO] fast_kafka_api.application: consumers_async_loop(): Kafka admin created <confluent_kafka.admin.AdminClient object at 0x7fac996e7cc0>.
[DEBUG] fast_kafka_api.confluent_kafka: create_missing_topics(['hello']): existing_topics=['prediction_request', '__consumer_offsets', 'hello', 'prediction_status', 'training_request', 'training_status', 'hello_world'], num_partitions=3, replication_factor=3
[INFO] fast_kafka_api.application: consumers_async_loop(): Kafka topics ['hello'] created if needed.
%4|1670831653.326|CONFWARN|rdkafka#producer-2| [thrd:app]: Configuration property group.id is a consumer property and will be ignored by this producer instance
[INFO] fast_kafka_api.application: AIOProducer created.
[INFO] fast_kafka_api.application: consumers_async_loop(topic=hello, config={'bootstrap.servers': 'tvrtko-fast-kafka-api-kafka-1:9092', 'group.id': 'hello_copnsumer_group'}, timeout=1.0) starting.
[INFO] fast_kafka_api.application: consumers_async_loop(topic=hello): Kafka Consumer for topic created.
[INFO] fast_kafka_api.application: consumers_async_loop(topic=hello): Kafka Consumer subscribed to topic.
[DEBUG] fast_kafka_api.application: _consumer_pooling_step()
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
```

Now you can interact with your consumer, by sending the messages to the subscribed 'hello' topic, don't worry, we will cover this in the next step of this guide.

## Recap

In this guide we have:
    
    1. Created a simple Kafka consumer using FastKafkaAPI
    2. Sent a message to our consumer trough Kafka
    3. Created a simple Kafka producer using FastKafkaAPI